In [15]:
import os

In [16]:
%pwd

'C:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidney'

In [17]:
os.chdir("C:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidney")

In [18]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/quamrl-hoda/diseaseClassification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "quamrl-hoda"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "10bd4471835788590657bbf53e006492e901bc34"

In [19]:
import tensorflow as tf

In [20]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [21]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int

In [22]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories,save_json


In [23]:
class ConfigurationManager:
  def __init__(
      self,
      config_filepath:Path=CONFIG_FILE_PATH,
      params_filepath:Path=PARAMS_FILE_PATH
  ):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])
  
  def get_evaluation_config(self) -> EvaluationConfig:
    eval_config = EvaluationConfig(
      path_of_model=Path("artifacts/training/model.h5"),
      training_data=Path("artifacts/data_ingestion/content/drive/MyDrive/CT-kidney-data"),
      mlflow_uri="https://dagshub.com/quamrl-hoda/diseaseClassification.mlflow",
      all_params=self.params,
      params_image_size=self.params.IMAGE_SIZE,
      params_batch_size=self.params.BATCH_SIZE
    )
    return eval_config

In [24]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [25]:
class Evaluation:
  def __init__(self, config:EvaluationConfig):
    self.config = config

  def _valid_generator(self):

    datagenerator_kwargs = dict(
      rescale=1./255,
      validation_split=0.30 # 30% data for validation
    )
    dataflow_kwargs = dict(
      target_size=self.config.params_image_size[:-1],
      batch_size=self.config.params_batch_size,
      class_mode="categorical"
    )

    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

    self.valid_generator = valid_datagenerator.flow_from_directory(
      directory=self.config.training_data,
      subset="validation",
      shuffle=False,  
      **dataflow_kwargs
    )

  @staticmethod
  def load_model(path:Path) -> tf.keras.Model:
      model = tf.keras.models.load_model(path)
      return model
    
  def evaluation(self):
      self.model = self.load_model(self.config.path_of_model)
      self._valid_generator()
      self.score = self.model.evaluate(self.valid_generator)
      self.save_score()
  # def save_score(self):
  #     scores = {"loass":self.score[0], "accuracy":self.score[1]}
  #     save_json(Path("artifacts/evaluation/scores.json"), data=scores)
  def save_score(self):
    scores = {"loss": self.score[0], "accuracy": self.score[1]}
    # create folder if not exists
    os.makedirs("artifacts/evaluation", exist_ok=True)
    save_json(Path("artifacts/evaluation/scores.json"), data=scores)

  def log_into_mlflow(self):

    # Clean parameters for MLflow
    clean_params = {}
    for key, value in self.config.all_params.items():
        if isinstance(value, (list, tuple, dict)):
            clean_params[key] = str(value)
        else:
            clean_params[key] = value

    mlflow.log_params(clean_params)

    # Log metrics
    mlflow.log_metrics({
        "eval_loss": self.score[0],
        "eval_accuracy": self.score[1]
    })

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        mlflow.keras.log_model(
            self.model,
            "model",
            registered_model_name="VGG16Model"
        )
    else:
        mlflow.keras.log_model(self.model, "model")

In [26]:
try:
  config = ConfigurationManager()
  eval_config = config.get_evaluation_config()

  evaluation = Evaluation(config=eval_config)
  evaluation.evaluation()
  evaluation.save_score()
  evaluation.log_into_mlflow()
except Exception as e:
  raise e


[2026-01-04 17:05:08,225:INFO:common:yaml file: config\config.yaml loaded successfully]
[2026-01-04 17:05:08,234:INFO:common:yaml file: params.yaml loaded successfully]
[2026-01-04 17:05:08,234:INFO:common:created directory at: artifacts]
Found 3735 images belonging to 4 classes.
234/234 [==============================] - 691s 3s/step - loss: 11.5808 - accuracy: 0.3780
[2026-01-04 17:16:41,498:INFO:common:json file saved at: artifacts\evaluation\scores.json]
[2026-01-04 17:16:41,513:INFO:common:json file saved at: artifacts\evaluation\scores.json]


2026/01/04 17:16:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-04 17:16:50,407:WARNING:save:Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\quamr\AppData\Local\Temp\tmpwey_mfu7\model\data\model\assets
[2026-01-04 17:16:51,870:INFO:builder_impl:Assets written to: C:\Users\quamr\AppData\Local\Temp\tmpwey_mfu7\model\data\model\assets]


c:\Users\quamr\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2026/01/04 17:23:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
